In [221]:
import numpy as np
import cvxpy as cp
import networkx as nx

from numpy.random import default_rng

In [254]:
def decompose_psd(X):
    n = X.shape[0]
    eigen_values, eigen_vectors = np.linalg.eigh(X)
    non_neg_eig_idx = [i for i in range(n) if eigen_values[i] >= 0]
    return eigen_vectors[:, non_neg_eig_idx] @ np.diag(np.sqrt(eigen_values[non_neg_eig_idx]))

In [206]:
def normalize_rows(X):
    # normalize the rows of matrix X
    return X / np.linalg.norm(X, axis=1)[:, np.newaxis]

# Phase recovery

In [208]:
n = 20  # number of observations
p = 5  # dimension of x
max_val = 1e2

In [209]:
rng = default_rng()
A = rng.random((n,p)) * max_val + rng.random((n,p)) * max_val * 1j
assert np.linalg.matrix_rank(A, tol=1e-6) >= p  # A must be injective; if the rows of A are linearly independent, AA+ = I
b = rng.random(n) * max_val
M = np.diag(b) @ (np.identity(n) - A @ np.linalg.pinv(A)) @ np.diag(b)

\begin{equation*}
\begin{aligned}
\min \quad & u^{*}Mu \\
\textrm{s.t.} \quad & |u_{i}| = 1, \qquad i=1,...,n
\end{aligned}
\end{equation*}

In [210]:
# u = cp.Variable(n, complex=True)
# qp_constraints = [ u[i].value.H * u[i] == 1 for i in range(n) ]
# prob = cp.Problem(cp.Minimize(cp.quad_form(u, M)), qp_constraints)
# prob.solve()

\begin{equation*}
\begin{aligned}
\min \quad & M \cdot U \\
\textrm{s.t.} \quad & U_{ii} = 1, \qquad i=1,...,n \\
& U \succeq 0
\end{aligned}
\end{equation*}

In [211]:
U = cp.Variable((n,n), hermitian=True)

In [212]:
sdp_constraints = [U >> 0]
sdp_constraints += [U[i][i] == 1 for i in range(n)]

In [213]:
prob = cp.Problem(cp.Minimize(cp.real(cp.trace(M @ U))), sdp_constraints)
prob.solve()

print("The optimal value is", prob.value)
print("A solution U is", U.value)

The optimal value is 22.459773868576804
A solution U is [[ 1.        +0.j          0.2130896 +0.85194106j  0.76683857+0.57323576j
   0.87212047-0.25251582j  0.091191  -0.52391815j  0.57483274+0.64089092j
  -0.25609023-0.23442449j -0.60119617-0.33651438j -0.05755594-0.49834867j
   0.55673235+0.54762895j]
 [ 0.2130896 -0.85194106j  1.        +0.j          0.73530137-0.64112327j
   0.15753351-0.72412401j -0.0941659 +0.0416755j   0.89249614-0.25808351j
  -0.06907264-0.24032299j -0.5666692 +0.75212213j -0.23597255+0.30459843j
   0.87497152-0.28491649j]
 [ 0.76683857-0.57323576j  0.73530137+0.64112327j  1.        +0.j
   0.55729986-0.57723327j -0.21994835-0.20974872j  0.84427244+0.30432958j
  -0.066762  -0.09310152j -0.85917117+0.12735995j -0.43035973-0.12053347j
   0.81170791+0.26663879j]
 [ 0.87212047+0.25251582j  0.15753351+0.72412401j  0.55729986+0.57723327j
   1.        +0.j          0.55692439-0.3509648j   0.55260502+0.71056251j
  -0.14268849-0.66155418j -0.46435181-0.14256717j  0.3545

In [219]:
np.linalg.matrix_rank(U.value,1e-9)

5

### Hyperplane rounding

In [207]:
def complex_hyperplane_rounding(Y, cost, iter=100):
    min_cost = np.Inf
    best_x = None
    d = Y.shape[1]
    rng = default_rng()
    for i in range(iter):
        # r = rng.random((d,1)) + rng.random((d,1)) * 1j
        r = rng.normal(size=(d,1)) + rng.normal(size=(d,1)) * 1j
        x = normalize_rows(Y @ r)
        cost_val = cost(x)
        if cost_val < min_cost:
            min_cost = cost_val
            best_x = x
    return best_x, min_cost

In [216]:
qp_cost = lambda u: cp.real(cp.quad_form(u, M)).value

In [217]:
complex_hyperplane_rounding(decompose_psd(U.value), qp_cost)

(array([[-0.03729563-0.99930428j],
        [-0.85490558-0.51878362j],
        [-0.31758806-0.94822878j],
        [-0.52026777-0.85400319j],
        [-0.96077373-0.27733346j],
        [-0.87824693-0.47820741j],
        [ 0.46508029-0.88526851j],
        [-0.10401327+0.99457591j],
        [-0.92673749+0.37570949j],
        [-0.82388862-0.56675175j]]),
 array([[422.92709868]]))

# Max-cut (for reference)

In [290]:
data_path = "../dat/"
# graph_file = "torusg3-8.dat"
graph_file = "toruspm3-8-50.dat"

with open(data_path + graph_file) as inf:
    next(inf, '')   # skip first line
    G = nx.read_weighted_edgelist(inf, nodetype=int, encoding="utf-8")

In [291]:
first_vertex = np.floor(default_rng().random() * (len(G) - n - 1)).astype(int)
G = G.subgraph(range(first_vertex, first_vertex + n))
L = nx.laplacian_matrix(G).toarray() * 1.0

In [292]:
X = cp.Variable((n,n), PSD=True)

\begin{equation*}
\begin{aligned}
\max \quad & \frac{1}{4} L \cdot X \\
\textrm{s.t.} \quad & X_{ii} = 1, \qquad i=1,...,n \\
& X \succeq 0
\end{aligned}
\end{equation*}

In [293]:
constraints = [ X[i][i] == 1 for i in range(n) ]

In [294]:
prob = cp.Problem(cp.Maximize(1/4 * (cp.trace(L @ X))), constraints)
prob.solve()

38.383876279426225

### Hyperplane rounding

In [295]:
def hyperplane_rounding(Y, cost, iter=100):
    min_cost = 0
    best_x = None
    d = Y.shape[1]
    rng = default_rng()
    for i in range(iter):
        r = rng.random((d,1))
        r = r / np.linalg.norm(r)
        x = np.sign(Y @ r)
        cost_val = cost(x)
        if cost_val < min_cost:
            min_cost = cost_val
            best_x = x
    return best_x, min_cost

In [296]:
mc_cost = lambda x: -1/4 * (cp.quad_form(x, L)).value

In [297]:
hyperplane_rounding(decompose_psd(X.value), mc_cost)

(array([[-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [-1.]]),
 array([[-36.]]))